In [246]:
from sklearn.externals import joblib
clf = joblib.load('finalized_model.pkl') 

In [248]:
import pandas as pd
user_Data = pd.read_csv('user_data.csv', encoding= 'utf-8', )

In [249]:
user_Data = user_Data[['text']]

In [250]:
t = group_data['description'].to_string()

In [251]:
f = open('titles_list.txt')
doc1 = str(f.read())

In [252]:
text1 = group_data['description'].tolist()
text1 = '\n'.join(map(str, text1))

In [253]:
group_data.head()

,name,class,description
0,Clothing Swap And Events,1,"Clothing Swaps are fashionable, fun and philan..."
1,Santa Cruz Women's Clothing Swap,1,"Meet with locals in a women-only, relaxed and ..."
2,San Francisco Organic + Plant-Based Food Meetup,1,"Meet people who are interested in organic, raw..."
3,African Heritage Resources Network,1,"Connections, Innovation, Transformation, Total..."
4,Craft Labs,1,Get your creative juices flowing at Craft Labs...


In [254]:
def get_text_file(filename):
    group_data = pd.read_csv(filename)
    group_data.dropna(axis=1, how='all')
    text1 = group_data['description'].tolist()
    text1 = ' '.join(map(str, text1))
    return text1

In [258]:
#Category 2 top_text
group_data = pd.read_csv('group_data.csv')

In [259]:
train_set = []
def get_train_set(class_no, user_text):
    train_set = []
    train_set.append(user_text)
    cat = group_data[group_data['class'] == class_no]['description'].values.astype('U').tolist()    
    names = group_data[group_data['class'] == class_no]['name'].values.astype('U').tolist() 
    for text in cat:
        train_set.append(text)
    return train_set,names
train_set,event_l = get_train_set(0,user_text)

In [260]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
#train_set = [user_text,cat2[89],cat2[100], cat2[89]]
stopWords = stopwords.words('english')


In [261]:
len(train_set)
#len(event_l)

650

In [262]:
# Set up the vectoriser, passing in the stop words
tfidf_vectorizer = TfidfVectorizer(stop_words=stopWords)

# Apply the vectoriser to the training set
tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set)

# Print the score'
#print ("\nSimilarity Score [*] ",cosine_similarity(tfidf_matrix_train, tfidf_matrix_train))
l = cosine_similarity(tfidf_matrix_train, tfidf_matrix_train)[0]

In [263]:
ranking = [i[0] -1  for i in sorted(enumerate(l), key=lambda x:x[1], reverse=True)] 
ranking = ranking[1:]

In [264]:
recs = []
for i in ranking[:20]:
    recs.append(event_l[i])

In [265]:
recs

['The Zero Net Energy Working Group (ZNEWG)',
 'Infinity Qigong',
 'SOLAR 101: Benefits of Solar & How Solar Works',
 'Berkeley Clean Energy Hackers',
 'Napa Solar Power Vocational and Education Meetup',
 'Women in Cleantech & Sustainability',
 'Solar Meet-Up',
 'Energy Wellness Community',
 'Keep people out of your energy (personal boundaries)',
 'San Rafael Clean Energy Meetup',
 'CA Smart Grid Group',
 'MomBoss Healthy Weight Loss Meetup',
 'The Reiki Circle Exchange',
 'The San Francisco Reiki Meetup',
 'Urban Permaculture',
 'Software for Renewable Energy - Bay Area',
 'Real Energy Solutions',
 'The Microtransit Solution',
 'San Jose Ayurveda Networking Meetup',
 'IEEE YP & SCEA - Happy Hour!']

In [141]:
from __future__ import division
import string
import math

tokenize = lambda doc: doc.lower().split(" ")

def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

def term_frequency(term, tokenized_document):
    return tokenized_document.count(term)

def sublinear_term_frequency(term, tokenized_document):
    count = tokenized_document.count(term)
    if count == 0:
        return 0
    return 1 + math.log(count)

def augmented_term_frequency(term, tokenized_document):
    max_count = max([term_frequency(t, tokenized_document) for t in tokenized_document])
    return (0.5 + ((0.5 * term_frequency(term, tokenized_document))/max_count))

def inverse_document_frequencies(tokenized_documents):
    idf_values = {}
    all_tokens_set = set([item for sublist in tokenized_documents for item in sublist])
    for tkn in all_tokens_set:
        contains_token = map(lambda doc: tkn in doc, tokenized_documents)
        idf_values[tkn] = 1 + math.log(len(tokenized_documents)/(sum(contains_token)))
    return idf_values

def tfidf(documents):
    tokenized_documents = [tokenize(d) for d in documents]
    idf = inverse_document_frequencies(tokenized_documents)
    tfidf_documents = []
    for document in tokenized_documents:
        doc_tfidf = []
        for term in idf.keys():
            tf = sublinear_term_frequency(term, document)
            doc_tfidf.append(tf * idf[term])
        tfidf_documents.append(doc_tfidf)
    return tfidf_documents

#in Scikit-Learn
from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(train_set)
